In [1]:

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# prompt: set tensor flow for gpu

import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow is using GPU")
  # Set memory growth to prevent TensorFlow from allocating all available GPU memory
  for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
else:
  print("TensorFlow is not using GPU")

TensorFlow is not using GPU


In [4]:

csv_data ='/content/drive/MyDrive/Colab Notebooks/winrate/Final_level_data/lvls_simulated.csv'
data = pd.read_csv(csv_data, index_col=0)

data.head()

,r_weights,b_weights,g_weights,y_weights,moves,target.yellow.goal_0,v_weights,target.blue.goal_0,target.green.goal_0,target.red.goal_0,o_weights,target.orange.goal_0,w_weights,target.drop1.goal_0,c_weights,h_weights,target.flower.goal_0,p_weights,target.smoke.goal_0,z_weights,t_weights,target.tire.goal_0,frogs,bugs,winkies,target.munchkins.goal_0,x_weights,target.colorizer.goal_0,target.colorizeb.goal_0,target.colorizev.goal_0,target.colorizey.goal_0,target.colorizeg.goal_0,helprovers,e_weights,target.energy.goal_0,target.munchkinegg.goal_0,target.bouquet.goal_0,winrate
level,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,25,25,25,25,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,93.15
2,24,30,23,23,10,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,94.18
3,20,20,20,35,10,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,97.67
4,18,18,30,17,10,0,17,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95.32
5,18,18,18,28,10,21,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96.84


In [5]:
data.shape

(29270, 38)

In [6]:

from tensorflow.keras.callbacks import ModelCheckpoint

# Features and target
X = data.drop(columns=['winrate'])  # Replace with your feature columns
y = data['winrate']  # Replace with your target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [10]:
100*X_train.shape[0]//1000

2341

In [11]:

# Define the DNN model
def build_dnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dropout(0.3),  # Dropout for regularization
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',  # Mean Squared Error for regression
                  metrics=['mae'])  # Mean Absolute Error
    return model

# Build and train the model
model = build_dnn_model(X_train.shape[1])

print(model.summary())

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_4/model_epoch_{epoch:05d}.keras',  # Save model with the epoch number in the filename
    save_freq=  200 * X_train.shape[0] // 1000,  # Calculate the batch frequency for every 10,000 epochs
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=2000,  # Adjust based on convergence
    batch_size=256,
    callbacks=[ checkpoint_callback ,early_stopping], #
    verbose=1
)


# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │          19,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 203,905 (796.50 KB)

 Trainable params: 202,113 (789.50 KB)

 Non-trainable params: 1,792 (7.00 KB)

None
Epoch 1/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 568.5662 - mae: 19.8432 - val_loss: 503.1769 - val_mae: 18.6742
Epoch 2/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 481.7614 - mae: 18.0439 - val_loss: 387.2188 - val_mae: 15.4536
Epoch 3/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 343.5084 - mae: 14.2023 - val_loss: 248.3028 - val_mae: 10.8813
Epoch 4/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 213.9057 - mae: 10.0091 - val_loss: 160.2642 - val_mae: 8.0171
Epoch 5/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 171.0523 - mae: 8.6032 - val_loss: 142.0815 - val_mae: 7.5862
Epoch 6/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 147.6358 - mae: 8.1822 - val_loss: 134.2519 - val_mae: 7.5661
Epoch 7/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 148.2029 - mae: 8.2934 - val_loss: 132.3324 - val_mae: 7.5155
Epoch 8/2000
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 148.3435 - mae: 8.2598 - val_loss: 129.0493 - val_mae: 7.4547
Epoc

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_4/model_dnn4_01')

In [ ]:
import glob

In [ ]:
modelpaths = glob.glob('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/*.keras')

for modelpath in modelpaths:
  print(modelpath )
  model = tf.keras.models.load_model(modelpath)

    # Evaluate the model
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f"Mean Squared Error: {mse}")
  print(f"R^2 Score: {r2}")
  print('-------------------' )

/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00125.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Mean Squared Error: 185.95371713399263
R^2 Score: -0.054994972376511075
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00250.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Mean Squared Error: 168.68804249099378
R^2 Score: 0.04296058465022534
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00374.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Mean Squared Error: 171.9145593209872
R^2 Score: 0.02465517464608724
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00499.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Mean Squared Error: 180.2533405757398
R^2 Score: -0.02265429802862351
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00623.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Mean Squared Error: 189.16786635263264


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input

In [ ]:
# Step 1: Load the pre-trained model
model_path = '/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_01246.keras'
base_model = tf.keras.models.load_model(model_path)

# Step 2: Freeze some layers (optional)
# Uncomment the line below to freeze all layers
# for layer in base_model.layers:
#     layer.trainable = False

# Step 3: Add new layers (optional, for transfer learning)
# You can add layers on top of the base model

# Add an Input layer if the base model is Sequential
input_shape = (37,)  # Replace with the number of features in your dataset
inputs = Input(shape=input_shape)
x = base_model(inputs)  # Pass the inputs through the base model


# Add new layers for fine-tuning
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
new_output = Dense(1, activation='linear')(x)  # Output layer for regression

# Combine the new layers into a new model
model = Model(inputs=inputs, outputs=new_output)


# x = base_model.output  # Get the output of the pre-trained model
# x = Dense(128, activation='relu')(x)  # Add a new dense layer
# x = Dropout(0.3)(x)  # Add dropout to reduce overfitting
# new_output = Dense(1, activation='linear')(x)  # Final output layer for regression
# model = Model(inputs=base_model.input, outputs=new_output)  # Create the new model

# # Step 4: Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

base_model.summary(), model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │          19,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 557,573 (2.13 MB)

 Trainable params: 185,345 (724.00 KB)

 Non-trainable params: 1,536 (6.00 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 37)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 1)                   │         186,881 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 187,266 (731.51 KB)

 Trainable params: 185,730 (725.51 KB)

 Non-trainable params: 1,536 (6.00 KB)

(None, None)

In [ ]:


# Features and target
X = data.drop(columns=['winrate'])  # Replace with your feature columns
y = data['winrate']  # Replace with your target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# # Example dataset
# data = pd.read_csv('/path/to/new_data.csv')  # Replace with your dataset path
# X = data.drop(columns=['winrate'])  # Features
# y = data['winrate']  # Target variable

# # Train-test split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Normalize the features (same as used during original training)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)

# # Step 6: Train the model on the new dataset
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=50,  # Number of epochs
#     batch_size=32,  # Batch size
#     verbose=1
# )

In [ ]:
model.summary()

Model: "functional_310"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 37)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 1)                   │         186,881 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 558,728 (2.13 MB)

 Trainable params: 185,730 (725.51 KB)

 Non-trainable params: 1,536 (6.00 KB)

 Optimizer params: 371,462 (1.42 MB)

In [ ]:

# from tensorflow.keras.callbacks import ModelCheckpoint

# # Early stopping to prevent overfitting
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# # Define the ModelCheckpoint callback
# checkpoint_callback = ModelCheckpoint(
#     filepath='/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_1/model_epoch_{epoch:05d}.keras',  # Save model with the epoch number in the filename
#     save_freq=  200 * X_train.shape[0] // 32,  # Calculate the batch frequency for every 10,000 epochs
#     verbose=1
# )

# # # Step 6: Train the model on the new dataset
# # history = model.fit(
# #     X_train, y_train,
# #     validation_data=(X_test, y_test),
# #     epochs=50,  # Number of epochs
# #     batch_size=32,  # Batch size
# #     verbose=1
# # )

# history = model.fit(
#     X_train, y_train,
#     validation_split=0.2,
#     epochs=5000,
#     batch_size=16,
#     callbacks=[checkpoint_callback], #early_stopping,
#     verbose=1
# )

# # Evaluate the model
# # y_pred = model.predict(X_test)
# # mse = mean_squared_error(y_test, y_pred)
# # r2 = r2_score(y_test, y_pred)

# # print(f"Mean Squared Error: {mse}")
# # print(f"R^2 Score: {r2}")

Epoch 1/5000


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# base_model.layers[0:-1]

In [ ]:
# Step 5: Load and preprocess your new dataset
# Load your new dataset (replace with your dataset path)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Example dataset
data = pd.read_csv('/path/to/new_data.csv')  # Replace with your dataset path
X = data.drop(columns=['winrate'])  # Features
y = data['winrate']  # Target variable

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features (same as used during original training)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Step 6: Train the model on the new dataset
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,  # Number of epochs
    batch_size=32,  # Batch size
    verbose=1
)

# Step 7: Save the updated model (optional)
model.save('/path/to/updated_model')

In [ ]:
# Step 1: Load the model
model_path = '/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN/model_epoch_00125.keras'  # Replace with your saved model's path
model = tf.keras.models.load_model(model_path)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Mean Squared Error: 185.95371713399263
R^2 Score: -0.054994972376511075


In [ ]:
# Step 1: Load the model
model_path = '/path/to/saved/model'  # Replace with your saved model's path
model = tf.keras.models.load_model(model_path)

# Step 2: Load or prepare input data
# Example: Assuming you have a CSV file with new data
data = pd.read_csv('/path/to/new_data.csv')  # Replace with your input data file path

# Extract features (assuming the target column is not present in the new data)
X_new = data.values

# Normalize/scale the input data (same scaler used during training)
scaler = StandardScaler()  # Use the scaler saved during training
X_new_scaled = scaler.fit_transform(X_new)

# Step 3: Make predictions
predictions = model.predict(X_new_scaled)

# Step 4: Display the predictions
print("Predictions:", predictions)